In [39]:
# Import necessary libraries
import pandas as pd

We want to clean and combine all of our various datasets and export it as one CSV. This will primarily involve dropping columns that we aren't interested in and merging onto a central Pandas DataFrame. After some exploratory analysis, we will determine what rows will beed to be dropped or interpolated.

The vaccine hesitancy dataset has values for multiple segments (ethnicity, social vulnerability, vaccine hesitancy), which we will split out into separate variables and look at each. Our primary index of county codes is given by Federal Information Processing Standards (FIPS).

In [48]:
vaccine_hesitancy = pd.read_csv('datasets/raw/Vaccine_Hesitancy_for_COVID-19__County_and_local_estimates.csv').rename(columns = {'FIPS Code':'fips'})
fips = vaccine_hesitancy[['fips']]
fips

,fips
0,1131
1,1129
2,1133
3,1127
4,2013
...,...
3137,55079
3138,55121
3139,56001
3140,55067


Percentage of ethnicity for each county are given. For readability and simplicity, we rename them with the most abundant ethnic group as primary and assume non-Hispanic for all of the non-Hispanic groups.

In [49]:
ethnicity = vaccine_hesitancy[['fips', 'Percent Hispanic', 'Percent non-Hispanic American Indian/Alaska Native', 'Percent non-Hispanic Asian', 'Percent non-Hispanic Black', 'Percent non-Hispanic Native Hawaiian/Pacific Islander', 'Percent non-Hispanic White']].rename(columns = {'Percent Hispanic': 'ethnicity_hispanic', 'Percent non-Hispanic American Indian/Alaska Native': 'ethnicity_native', 'Percent non-Hispanic Asian': 'ethnicity_asian', 'Percent non-Hispanic Black': 'ethnicity_black', 'Percent non-Hispanic Native Hawaiian/Pacific Islander': 'ethnicity_hawaiian', 'Percent non-Hispanic White': 'ethnicity_white'})
ethnicity

,fips,ethnicity_hispanic,ethnicity_native,ethnicity_asian,ethnicity_black,ethnicity_hawaiian,ethnicity_white
0,1131,0.0053,0.0009,0.0003,0.6938,0.0000,0.2684
1,1129,0.0146,0.0731,0.0025,0.2354,0.0000,0.6495
2,1133,0.0315,0.0034,0.0016,0.0073,0.0005,0.9370
3,1127,0.0249,0.0015,0.0049,0.0617,0.0000,0.8895
4,2013,0.0901,0.4588,0.1968,0.0322,0.0100,0.1321
...,...,...,...,...,...,...,...
3137,55079,0.1500,0.0047,0.0428,0.2606,0.0002,0.5124
3138,55121,0.0840,0.0034,0.0043,0.0051,0.0000,0.8953
3139,56001,0.0953,0.0091,0.0327,0.0150,0.0003,0.8248
3140,55067,0.0197,0.0069,0.0022,0.0125,0.0002,0.9383


In [50]:
social_vulnerability_index = vaccine_hesitancy[['fips', 'Social Vulnerability Index (SVI)', 'County Name', 'State']].rename(columns= {'Social Vulnerability Index (SVI)': 'social_vulnerability_index', 'County Name': 'county_name', 'State': 'state'})
social_vulnerability_index

,fips,social_vulnerability_index,county_name,state
0,1131,0.93,"Wilcox County, Alabama",ALABAMA
1,1129,0.73,"Washington County, Alabama",ALABAMA
2,1133,0.70,"Winston County, Alabama",ALABAMA
3,1127,0.75,"Walker County, Alabama",ALABAMA
4,2013,0.58,"Aleutians East Borough, Alaska",ALASKA
...,...,...,...,...
3137,55079,0.81,"Milwaukee County, Wisconsin",WISCONSIN
3138,55121,0.28,"Trempealeau County, Wisconsin",WISCONSIN
3139,56001,0.25,"Albany County, Wyoming",WYOMING
3140,55067,0.35,"Langlade County, Wisconsin",WISCONSIN


In [51]:
vaccine_hesitancy = vaccine_hesitancy[['fips', 'Estimated hesitant', 'Estimated strongly hesitant']].rename(columns = {'Estimated hesitant': 'vaccine_hesitant', 'Estimated strongly hesitant': 'vaccine_hesitant_strong'})
vaccine_hesitancy

,fips,vaccine_hesitant,vaccine_hesitant_strong
0,1131,0.23,0.11
1,1129,0.23,0.11
2,1133,0.22,0.11
3,1127,0.23,0.11
4,2013,0.26,0.12
...,...,...,...
3137,55079,0.18,0.11
3138,55121,0.18,0.10
3139,56001,0.30,0.16
3140,55067,0.17,0.10


We keep four columns from the education dataset representing percentage of the entire adult population with specific educational attainment signifiers and make sure to represent all the percentages as decimal values.

In [41]:
education = pd.read_csv('datasets/raw/Education.csv')
education = education[['FIPS Code', 'Percent of adults with less than a high school diploma, 2015-19', 'Percent of adults with a high school diploma only, 2015-19', "Percent of adults completing some college or associate's degree, 2015-19", "Percent of adults with a bachelor's degree or higher, 2015-19"]]
education = education.rename(columns = {'FIPS Code': 'fips', 'Percent of adults with less than a high school diploma, 2015-19': 'education_high_school_less', 'Percent of adults with a high school diploma only, 2015-19': 'education_high_school_only', "Percent of adults completing some college or associate's degree, 2015-19": 'education_degree_some', "Percent of adults with a bachelor's degree or higher, 2015-19": 'education_bachelors_degree'})
education_cols = ['education_high_school_less', 'education_high_school_only', 'education_degree_some', 'education_bachelors_degree']
education[education_cols] = education[education_cols].div(100)
education

,fips,education_high_school_less,education_high_school_only,education_degree_some,education_bachelors_degree
0,0,0.120,0.270,0.289,0.321
1,1000,0.138,0.308,0.299,0.255
2,1001,0.115,0.336,0.284,0.266
3,1003,0.092,0.277,0.313,0.319
4,1005,0.268,0.356,0.260,0.116
...,...,...,...,...,...
3278,72145,0.284,0.262,0.241,0.212
3279,72147,0.288,0.392,0.140,0.180
3280,72149,0.220,0.384,0.197,0.199
3281,72151,0.290,0.257,0.272,0.180


From the Poverty Estimate dataset, we keep the column of values for the percentage of the entire population that is in poverty in 2019. The data is in a narrow format, so we use pivot and make sure to represent the percentage as a decimal.

In [42]:
poverty = pd.read_csv('datasets/raw/PovertyEstimates.csv')
poverty = poverty[['FIPStxt', 'Attribute', 'Value']].pivot(index='FIPStxt', columns='Attribute', values='Value').reset_index()
poverty = poverty[['FIPStxt', 'PCTPOVALL_2019']].rename(columns = {'FIPStxt':'fips', 'PCTPOVALL_2019': 'poverty'})
poverty['poverty'] = poverty['poverty'].div(100)
poverty

Attribute,fips,poverty
0,0,0.123
1,1000,0.156
2,1001,0.121
3,1003,0.101
4,1005,0.271
...,...,...
3188,56037,0.083
3189,56039,0.060
3190,56041,0.085
3191,56043,0.111


In [43]:
natality = pd.read_csv('datasets/raw/Natality, 2016-2019 expanded.txt', sep='\t')
natality
# TODO: get full county birth rate data

,Notes,County of Residence,County of Residence Code,Births,Total Population,Birth Rate
0,NaN,"Baldwin County, AL",1003.0,2329.0,223234.0,10.433
1,NaN,"Calhoun County, AL",1015.0,1383.0,113605.0,12.174
2,NaN,"Etowah County, AL",1055.0,1236.0,102268.0,12.086
3,NaN,"Houston County, AL",1069.0,1336.0,105882.0,12.618
4,NaN,"Jefferson County, AL",1073.0,8481.0,658573.0,12.878
...,...,...,...,...,...,...
666,"7. Bronx, New York (FIPS code 36005) represent...",NaN,NaN,NaN,NaN,NaN
667,"8. Kings County, New York (FIPS code 36047) re...",NaN,NaN,NaN,NaN,NaN
668,"9. New York County, New York (FIPS code 36061)...",NaN,NaN,NaN,NaN,NaN
669,"10. Queens, New York (FIPS code 36081) represe...",NaN,NaN,NaN,NaN,NaN


In [44]:
elections = pd.read_csv('datasets/raw/countypres_2000-2016.tab', sep='\t')
elections = pd.melt(elections, id_vars='FIPS', var_name='party', value_vars='party')
elections['FIPS'].unique()
elections.groupby(['FIPS', 'value']).count()
# TODO: check amount if results are available at all county levels

party
FIPS    value            
1001.0  democrat        5
        green           1
        republican      5
1003.0  democrat        5
        green           1
...                   ...
56043.0 green           1
        republican      5
56045.0 democrat        5
        green           1
        republican      5

[9464 rows x 1 columns]

From the Unemployment dataset, we have several useful data points involving geography (rural vs urban continuum code, urban influence code), income (median household, and represented as a percent of median state total) and unemployment rate.

In [54]:
unemployment = pd.read_csv('datasets/raw/Unemployment.csv').pivot(index='fips_txt', columns='Attribute', values='Value').reset_index().rename(columns = {'fips_txt':'fips'})
geography = unemployment[['fips', 'Rural_urban_continuum_code_2013', 'Urban_influence_code_2013']].rename(columns={'Rural_urban_continuum_code_2013': 'rural_urban_code', 'Urban_influence_code_2013': 'urban_influence_code'})
# TODO: convert urban/rural codes into z-scores
geography

Attribute,fips,rural_urban_code,urban_influence_code
0,0,NaN,NaN
1,1000,NaN,NaN
2,1001,2.0,2.0
3,1003,3.0,2.0
4,1005,6.0,6.0
...,...,...,...
3270,72145,1.0,1.0
3271,72147,7.0,12.0
3272,72149,2.0,2.0
3273,72151,1.0,1.0


To look at the county's economic factors, we keep two columns representing the estimated median household income in 2019 and the county household median income as a percent of the state total median household income. We represent this percent as a decimal.

In [55]:
income = unemployment[['fips', 'Med_HH_Income_Percent_of_State_Total_2019', 'Median_Household_Income_2019']].rename(columns={'Med_HH_Income_Percent_of_State_Total_2019': 'median_income_percent_state', 'Median_Household_Income_2019': 'median_income'})
income['median_income_percent_state'] = income['median_income_percent_state'].div(100)
income

Attribute,fips,median_income_percent_state,median_income
0,0,NaN,65712.0
1,1000,1.000000,51771.0
2,1001,1.124819,58233.0
3,1003,1.156458,59871.0
4,1005,0.694829,35972.0
...,...,...,...
3270,72145,NaN,NaN
3271,72147,NaN,NaN
3272,72149,NaN,NaN
3273,72151,NaN,NaN


In [56]:
unemployment = unemployment[['fips', 'Unemployment_rate_2019']].rename(columns={'Unemployment_rate_2019': 'unemployment'})
unemployment['unemployment'] = unemployment['unemployment'].div(100)
unemployment

Attribute,fips,unemployment
0,0,0.036694
1,1000,0.030000
2,1001,0.027000
3,1003,0.027000
4,1005,0.038000
...,...,...
3270,72145,0.096000
3271,72147,0.069000
3272,72149,0.159000
3273,72151,0.131000


We set the fips as index for all of our dataframes and then concatenate them along it with an inner join. We note that there's only one row that was lost.

In [31]:
dfs = [df.set_index('fips') for df in [vaccine_hesitancy, social_vulnerability_index, ethnicity, unemployment, geography, income, poverty, education]]
df = pd.concat(dfs, axis=1, join='inner').reset_index()
df

,fips,vaccine_hesitant,vaccine_hesitant_strong,social_vulnerability_index,county_name,state,ethnicity_hispanic,ethnicity_native,ethnicity_asian,ethnicity_black,...,unemployment,rural_urban_code,urban_influence_code,median_income_percent_state,median_income,poverty,education_high_school_less,education_high_school_only,education_degree_some,education_bachelors_degree
0,1131,0.23,0.11,0.93,"Wilcox County, Alabama",ALABAMA,0.0053,0.0009,0.0003,0.6938,...,0.071,9.0,10.0,0.598752,30998.0,0.325,0.235,0.395,0.245,0.125
1,1129,0.23,0.11,0.73,"Washington County, Alabama",ALABAMA,0.0146,0.0731,0.0025,0.2354,...,0.046,8.0,7.0,0.943849,48864.0,0.186,0.174,0.431,0.269,0.127
2,1133,0.22,0.11,0.70,"Winston County, Alabama",ALABAMA,0.0315,0.0034,0.0016,0.0073,...,0.033,6.0,4.0,0.788608,40827.0,0.167,0.212,0.382,0.278,0.128
3,1127,0.23,0.11,0.75,"Walker County, Alabama",ALABAMA,0.0249,0.0015,0.0049,0.0617,...,0.033,1.0,1.0,0.888354,45991.0,0.173,0.182,0.375,0.330,0.113
4,2013,0.26,0.12,0.58,"Aleutians East Borough, Alaska",ALASKA,0.0901,0.4588,0.1968,0.0322,...,0.028,9.0,12.0,0.866845,66923.0,0.148,0.145,0.435,0.305,0.115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3136,55079,0.18,0.11,0.81,"Milwaukee County, Wisconsin",WISCONSIN,0.1500,0.0047,0.0428,0.2606,...,0.040,1.0,1.0,0.833710,53505.0,0.169,0.117,0.284,0.290,0.310
3137,55121,0.18,0.10,0.28,"Trempealeau County, Wisconsin",WISCONSIN,0.0840,0.0034,0.0043,0.0051,...,0.036,6.0,6.0,0.961201,61687.0,0.089,0.093,0.390,0.324,0.193
3138,56001,0.30,0.16,0.25,"Albany County, Wyoming",WYOMING,0.0953,0.0091,0.0327,0.0150,...,0.031,4.0,5.0,0.789334,52216.0,0.160,0.041,0.147,0.295,0.518
3139,55067,0.17,0.10,0.35,"Langlade County, Wisconsin",WISCONSIN,0.0197,0.0069,0.0022,0.0125,...,0.042,6.0,6.0,0.772816,49597.0,0.130,0.097,0.423,0.313,0.167


In [57]:
df.to_csv('datasets/clean/interim_clean_dataset_2021-05-21.csv')